# In this Note Book we will be going over the Bitcoin POW(proof of work) algorithm with python.

### The Bitcoin POW algorithm is one of the key steps taken that secures the network, along with public and private key signatures on transactions.

### The BTC POW algorithm is slow to compute, but fast to verify, and thats what makes it work. BTC POW uses SHA256 as its hashing algorithm

#### First lets take a look at hashing.  Hashing is a function that maps data to a *unique key.  This data can be anything, whether that be a file, or in our example, a character string. In BTC it is the block.

*some hashing algorithms have what is called collisions.  This is where the hash of data input gives the same hash as another data input.  This is currently understood to not be mathetmatically possible with SHA256

For demonstration, lets hash something with sha256.  These outputs, while not random are impossible to predict without already having hashed them.

In [ ]:
import hashlib
message = "Hello world"        
message = str.encode(message)
hashed_message = hashlib.sha256(message).hexdigest()
print(hashed_message)

#Even just changing/adding/removing/moving one character will give a unique hash:

message = "Hello World"        
message = str.encode(message)
hashed_message = hashlib.sha256(message).hexdigest()
print(hashed_message)

message = "World Hello"        
message = str.encode(message)
hashed_message = hashlib.sha256(message).hexdigest()
print(hashed_message)

### In BTC, blocks contain transaction data, as well as the hash of the previous block.  It also includes the timestamp, the nonce and a few more things.  For simplicity sake, in this video we will just be looking at the nonce

# Lets now import the rest of the code and then review it as needed.

In [ ]:
import hashlib
import argparse
import multiprocessing
import numpy as np
import time

#set the number of leading zeros we need to have
def set_difficulty(difficulty):
    check_hash = ""
    for i in range(difficulty):
        check_hash = check_hash + "0"
    return check_hash

#does POW on one core
def one_core(difficulty, base_message,process_size):
    counter = 0
    solution = set_difficulty(difficulty)
    start = time.time()
    while True:
        message = base_message + " " + str(counter)
        message = str.encode(message)
        hashed_message = hashlib.sha256(message).hexdigest()
        check_hash = hashed_message[:difficulty]
        if check_hash == solution:
            print()
            print(base_message + " " + str(counter))
            print()
            print(hashed_message)
            print("Hashed " + str(counter) + " times")
            break
        counter = counter + 1
        if counter % process_size == 0:
            end = time.time()
            time_elasped = end - start
            start = time.time()
            hash_rate = calc_hashrate(process_size,1,time_elasped)
            million = counter / 1000000
            print("Hashed " + str(int(million)) + " million times")
            print("Hashrate: " + str(hash_rate) + " MH/sec")

def multiprocess_hashing(difficulty,base_message,process_size,number_of_processes):
    counter = 0
    #gets the difficulty
    solution = set_difficulty(difficulty)
    #used for multiprocessing
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    exit_status = False
    while True:
        jobs = []
        #tracks time start
        start = time.time()
        for i in range(number_of_processes):
            #chunks works for each core
            low_bound = counter*process_size
            high_bound = ((counter+1)*process_size)-1

            counter = counter + 1
            #creates and starts job
            p = multiprocessing.Process(target=hash_process, args=(
                low_bound, high_bound, base_message, difficulty, solution, return_dict))
            jobs.append(p)
            p.start()
        #waits for each job to finish
        for proc in jobs:
            proc.join()
            status = return_dict.values()[0]
            #checks if a nounce was returned or not
            if status != False and exit_status == False:
                exit_status = True
                final_status = status
        #tracks the end time
        end = time.time()
        #prints the found nouce and the hashed data
        if exit_status:
            print("found nounce: " + base_message + " " + str(final_status))
            message = base_message + " " + str(final_status)
            message = str.encode(message)
            hashed_message = hashlib.sha256(message).hexdigest()
            print(hashed_message)
            break

        #status updates
        if process_size*number_of_processes>=1000000000:
            billions = high_bound/1000000000
            print("Hashed " + str(np.round(billions)) + " billion times")
            hashes = high_bound - low_bound
            time_elasped = end-start
            hashrate = calc_hashrate(hashes,number_of_processes,time_elasped)
            print("Hashrate: " + str(hashrate) + " MH/sec")
        else:
            millions = high_bound/1000000
            print("Hashed " + str(np.round(millions)) + " million times")
            hashes = high_bound - low_bound
            time_elasped = end-start
            hashes = high_bound - low_bound
            time_elasped = end-start
            hashrate = calc_hashrate(hashes,number_of_processes,time_elasped)
            print("Hashrate: " + str(hashrate) + " MH/sec")

#calculates MH/sec
def calc_hashrate(hashes,number_of_processes,time):
    hash_rate = hashes/1000000
    hash_rate = hash_rate * number_of_processes
    hash_rate = hash_rate/time
    hash_rate = np.round(hash_rate,3)
    return hash_rate


#process that does POW on more cores in sequential chunks
def hash_process(low_bound, high_bound, base_message, difficulty, solution, return_dict):
    return_dict['status'] = False
    for i in range(low_bound, high_bound):
        message = base_message + " " + str(i)
        message = str.encode(message)
        hashed_message = hashlib.sha256(message).hexdigest()
        check_hash = hashed_message[:difficulty]
        if check_hash == solution:
            return_dict['status'] = i
            break


    return return_dict

#runs algorithm and takes args
def main(base_message="Hello world",process_size=7,number_of_processes=10,difficulty=7,mode_to_run=0):
    #sets the number of hashes to do per process
    process_size = pow(10,process_size)

    if mode_to_run == 0:
        one_core(difficulty, base_message,process_size)
    else:
        multiprocess_hashing(difficulty, base_message,process_size,number_of_processes)

### So as its implied in the name, BTC POW is to prove the work has been done on a block. Work in this case is computational work.  This computational work is done by hashing a block until a sha256 hash is found that meets certain requirements. 

#### In Bitcoin, one of those requirements is that the block hash is below a certain value.  In other words, it is a requirement that a hash of some data lead with a certain amount of zeros. This is the concept of difficulty in the blockchain, with more leading zeros requiring more hashin(more work) to find. Lets look a demo.

In [27]:
#Using code we loaded earlier, we can set a diffculty string to check for
difficulty = 1
difficulty_string = set_difficulty(difficulty)
print(difficulty_string)
#If we set the difficulty to a higher level, say 3, we will checking for three leading zeros
difficulty = 3
difficulty_string = set_difficulty(difficulty)
print(difficulty_string)

0
000


#### As we saw previously, the hash for a string like "Hello world" is 64ec88ca00b268e5ba1a35678a1b5316d212f4f366b2477232534a8aeca37f3c

#### If we wanted to make it so that the string "Hello world" mets the requirements of a certain difficulty, we are going to have to add or change some data.  We don't want to change the message, so we are going to add a number, called a nonce to the end of the message and hash the entire thing.  We will keep hashing until we have a hash that meets the requirements.  Lets first take a look at what this means though

In [28]:
#Lets hash "Hello world" 5 times, each with a different nonce.
base_message = "Hello world"
for i in range(5):
  message = base_message + " " + str(i)
  print("Message to hash: ",message)
  message = str.encode(message)
  hashed_message = hashlib.sha256(message).hexdigest()
  print("Hashed message: ",hashed_message)
  print()

Message to hash:  Hello world 0
Hashed message:  835e01a61fdccc4a26d5aaf06b3eaded5018e3c23e9aa37371d9d6df0c324827

Message to hash:  Hello world 1
Hashed message:  f2f8c4dfc31be22607ea3d49a247b98fc62e6fd35429344d28f7be6fda187a9d

Message to hash:  Hello world 2
Hashed message:  995d462952b37ff30bc1a20b187ed90fc5359bcd55ad17c5a4d0fc6b88ca1153

Message to hash:  Hello world 3
Hashed message:  bb1c79418c616acb0e3c82a39cf2db264ec30cf3541e4e896e1a1477bc40b980

Message to hash:  Hello world 4
Hashed message:  96871d0bbe52dcb9246b2ec8e5436eb36ef71c22e1aeb196d5551230bda6fe28



#### As we can see, by adding changing the nonce we are changing the output of hash.  If we do this enough times, we should find a hash that starts with the needed number of zeros depending on the difficulty of the blockchain.

#### **Important to note with respect to Bitcoin, is that the nonce can only be up to 32 bits, or roughly 4 billion.  If this was the only variable that could be changed, this would causes issues as there may not be hash that exists with only 4 billion attempts(if the diffculty is high enough). However, Bitcoin contains transactions, and these tranactions can be in any order, meaning that the number of possible combinations with just the nonce and the tranactions(and there is more to a block as well like timestamp) is n!*2^32  which while not infinite, with even only a couple dozen tranactions, it might as well be**

#### Lets now briefly see in action how difficulty affects the total amount of work done from hashing and by extension the total amount of time

In [29]:
%%time
#difficulty of 1
main(base_message="Hello world",process_size=6,number_of_processes=1,difficulty=1,mode_to_run=0)


Hello world 31

0a39c74837af783db9302d7f621a721953b1f86a869100e49d74e58bbb4763bd
Hashed 31 times
CPU times: user 266 µs, sys: 0 ns, total: 266 µs
Wall time: 210 µs


In [30]:
%%time
#difficulty of 2
main(base_message="Hello world",process_size=6,number_of_processes=1,difficulty=2,mode_to_run=0)


Hello world 110

00a4b238a37aa3d043a75e5af6f44450e918ccff09ac1b4a6b65ec33493a2cfe
Hashed 110 times
CPU times: user 297 µs, sys: 1 µs, total: 298 µs
Wall time: 255 µs


In [31]:
%%time
#difficulty of 3
main(base_message="Hello world",process_size=6,number_of_processes=1,difficulty=3,mode_to_run=0)


Hello world 734

000e00fc5734ce3a04437d1ca3eecc05bb3b592480b5b29b77d1426f583ae868
Hashed 734 times
CPU times: user 1.25 ms, sys: 0 ns, total: 1.25 ms
Wall time: 1.2 ms


In [32]:
%%time
#difficulty of 4
main(base_message="Hello world",process_size=6,number_of_processes=1,difficulty=4,mode_to_run=0)


Hello world 17626

00003aabd6484a57810b931cb7bb9d9bce5f9b6e5115f3b482b46767a3aacea3
Hashed 17626 times
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.8 ms


In [33]:
%%time
#difficulty of 5
main(base_message="Hello world",process_size=6,number_of_processes=1,difficulty=5,mode_to_run=0)

Hashed 1 million times
Hashrate: 0.999 MH/sec
Hashed 2 million times
Hashrate: 1.017 MH/sec
Hashed 3 million times
Hashrate: 1.003 MH/sec
Hashed 4 million times
Hashrate: 1.004 MH/sec

Hello world 4304875

00000584a225dd625f432686cc78258639ccc97162454b6745bb363a8cffaa31
Hashed 4304875 times
CPU times: user 4.26 s, sys: 7.99 ms, total: 4.27 s
Wall time: 4.28 s


In [34]:
%%time
#difficulty of 6
main(base_message="Hello world",process_size=7,number_of_processes=1,difficulty=6,mode_to_run=0)

Hashed 10 million times
Hashrate: 0.983 MH/sec
Hashed 20 million times
Hashrate: 0.979 MH/sec

Hello world 26476165

000000690d1f0e3d780d62c8f681de53e87ee1f054555d03b1ab6ef0e04a9de2
Hashed 26476165 times
CPU times: user 27 s, sys: 0 ns, total: 27 s
Wall time: 27 s


In [35]:
%%time
#difficulty of 7, this one will take over several minutes on colab
main(base_message="Hello world",process_size=7,number_of_processes=1,difficulty=7,mode_to_run=0)

Hashed 10 million times
Hashrate: 0.981 MH/sec
Hashed 20 million times
Hashrate: 0.985 MH/sec
Hashed 30 million times
Hashrate: 0.987 MH/sec
Hashed 40 million times
Hashrate: 0.986 MH/sec
Hashed 50 million times
Hashrate: 0.996 MH/sec
Hashed 60 million times
Hashrate: 1.001 MH/sec
Hashed 70 million times
Hashrate: 0.989 MH/sec
Hashed 80 million times
Hashrate: 0.988 MH/sec
Hashed 90 million times
Hashrate: 0.995 MH/sec
Hashed 100 million times
Hashrate: 0.986 MH/sec
Hashed 110 million times
Hashrate: 0.984 MH/sec
Hashed 120 million times
Hashrate: 0.984 MH/sec
Hashed 130 million times
Hashrate: 0.984 MH/sec
Hashed 140 million times
Hashrate: 0.975 MH/sec
Hashed 150 million times
Hashrate: 0.998 MH/sec
Hashed 160 million times
Hashrate: 0.984 MH/sec
Hashed 170 million times
Hashrate: 0.978 MH/sec
Hashed 180 million times
Hashrate: 0.983 MH/sec
Hashed 190 million times
Hashrate: 0.99 MH/sec
Hashed 200 million times
Hashrate: 0.981 MH/sec
Hashed 210 million times
Hashrate: 0.981 MH/sec
Ha

#### So as one can see, for unique data, in order to find a hash that meets the diffculty requirements there is no way but to put in the work.

#### Important to note, is that once this work is done, it can be checked almost instantly

In [36]:
#Here we take a found nouce and message that has 7 leading zeros;  We can hash it and check it almost instantly
message = "Hello world 242124683"        
message = str.encode(message)
hashed_message = hashlib.sha256(message).hexdigest()
print(hashed_message)

0000000d215bc8336a60193e69d0517ef2b256be40c5029dff0637535db8b58c


#### So Bitcoin validators recieve work from miners and can quickly check the result.  A valid result will be accepted, and the mined block will be propogated to the network.  The miner will then recieve Bitcoin for this work.

#### By using POW, along with a few other methods, Bitcoin can be highly secure, distributed, immutable network.  One thing worth pointing out, is that since the previous block hash is one of the inputs in a blockchain, this POW makes it so one can not change past blocks without being detected and without redoing past work.